Dit notebook laat een voorbeeld zien van een evolutionary algorithm in de praktijk. Laad eerst nodige packages in

In [0]:
import random as rd
import numpy as np
import time

Het N-queens probleem is het probleem om N koninginnen dusdanig op een schaakbord van NxN neer te zetten, zodat geen enkele koningin een andere koningin kan gaan slaan. Voor de niet-schakers: Een koningin mag zowel verticaal, horizontaal als diagonaal bewegen, zoveel zetten als wenselijk. Hieronder een voorbeeld van een juiste oplossing voor N=8

> ![alt text](https://cdn-images-1.medium.com/max/1200/1*Zm2pbDR5CS2w2xeUbTBxQQ.png)

Voor N=8 zijn er in totaal 64 choose 8 = 4.426.165.368 mogelijke manieren om de koninginnen neer zetten, waarvan er slechts 92 een juiste oplossing zijn. Dit is dus al vrij onpraktisch om alle manieren af te gaan, laat staan als N groter is. Daarom kunnen we een evolutionary algorithm gebruiken om een oplossing te vinden.





Een evolutionary algorithm bestaat uit een aantal stappen dat iteratief doorlopen wordt, totdat een oplossing is bereikt die aan een bepaald criteria voldoet. Dit kan een vast aantal iteraties zijn, of totdat er een x aantal iteraties geen verbetering te zien is, of totdat een oplossing is bereikt (zoals in het N-queens probleem).


> ![alt text](https://cdn-images-1.medium.com/max/1600/1*odW0CYMTeS-R5WW1hM0NUw.jpeg)

Alvorens te beginnen is het goed om na te denken hoe we een oplossing willen representeren. In het geval van het N-queens probleem, kunnen we kiezen voor een 8x8 matrix, met een 1 als er een queen staat, en een 0 als er geen staat. Zoals bijvoorbeeld


```
[0, 0, 0, 0, 1, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 1, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
```


als in het schaakbord van hierboven. Dit geeft echter in totaal 2^64 mogelijkheden, dat zijn er wel erg veel om te kunnen doorlopen. We kunnen ook een stapje slimmer zijn: We nemen een 8x1 vector, met in iedere entry een getal tussen de 1 en de 8, die het rijnummer aangeven waarin de koningin staat. Zo zorgen we er automatisch voor dat er geen koninginnen op dezelfde kolommen staan. Nu zijn er nog maar 8^8 mogelijkheden over. Uiteraard kunnen we nog een stuk verder gaan: nu nemen we een 8x1 vector, met een iedere entry een getal tussen de 1 en de 8, waarbij ieder nummer maar een keer mag voorkomen: Zo kan er ook in iedere rij geen twee koninginnen staan en hebben we nog 8! = 40320 opties over, beduidend minder! Het bovenstaande voorbeeld heeft dan als representatie


```
[1, 7, 4, 6, 8, 2, 5, 3]
```

Zo'n representatie noemen we een individu. Voor andere problemen kun je ook andere representaties gebruiken, dit is afhankelijk van het probleem. Voor een evolutionary algorithm om te werken, heb je een hele populatie nodig, bestaande uit een heleboel individuen. De grootte van de populatie is een parameter, voor verschillende problemen werkt een andere populatie grootte. Daarnaast moeten we natuurlijk weten hoe "goed" een individu is. Daarvoor gebruiken we een fitness functie, in dit specifieke geval tellen we het aantal koninginnen die elkaar kunnen slaan.

---



Laten we beginnen met een class aan te maken voor een individu. We voegen twee methods toe: 


*   Een om random een individu te creeën
*   Een om de fitness van een individu te berekenen





In [0]:
# Maak class individual aan
class Individual:
    def __init__(self, length, genes = None):
        self.length = length
        self.fitness = None
        if genes is not None:
            self.genes = genes
        else:
            self.genes = []*length


    # Initialiseer de genen random
    def InitialiseRandom(self):
        self.genes = rd.sample(range(self.length), self.length)

    # Maak een fitness functie om te berekenen hoe goed de huidige individu is
    def GetFitness(self, N):
        """
        Berekent de fitness van een individual
        :param self: individual
        """
        fit = 0
        for i in range(N):
            """
            Tel het aantal queens dat deze queen kan slaan. 
            Door de encoding hoeft er alleen naar de diagonaal gekeken te worden
            Een queen kan geslagen worden door een andere queen als het absolute verschil in de genen gelijk
             is aan het absolute verschil in posities
            """
            for j in range(N):
                # Een queen kan zichzelf niet slaan
                if i != j:
                    if abs(self.genes[i] - self.genes[j]) == abs(i - j):
                        fit = fit + 1
        self.fitness = fit

Om te beginnen maken we dus een populatie aan van random aangemaakte individuen. Daarnaast zetten we een aantal parameters voor de grootte van het schaakbord en voor de grootte van de populatie

In [0]:
# Global parameters
population_size = 1000
N = 14 # De grootte van het schaakbord

# Initialiseer de populatie random
population = [Individual(N) for i in range(population_size)]
for i in population:
    # Initialise random and calculate the fitness for each individual
    i.InitialiseRandom()
    i.GetFitness(N)

Vervolgens moeten we bepalen welke individuen goed zijn. Betere individuen moeten namelijk een hogere kans te krijgen om "voor te planten", in analogie met in de natuur. Hiervoor gebruiken we de fitness functie.

Daarna zijn er verschillende manieren om ouders te selecteren die gebruikt worden om een nieuwe populatie te maken. Hieronder gebruiken we tournament selection: Er worden random een aantal individuen geselecteerd, en de "sterkste" inividu hiervan wordt toegevoegd aan de parent population. Dit wordt herhaald tot het aantal gewenste ouders is bereikt.

Andere manieren parent selection te doen zijn bijvoorbeeld:

*   Fitness Proportionate Selection
*   Rank Selection
*   Random Selection
*   Reward Based Selection
*   Age Based Selection
*   Elitism

Het is van het probleem afhankelijk welke manieren goed werken en welke minder

In [0]:
# Een voorbeeld van een parent selection mechanisme
def TournamentSelection(pop, size, tournament_size):
    """
    Geeft een populatie van parents op basis van tournament selection: Er worden tournament_size individuen geselecteerd
     uit pop waarvan degene met de hoogste fitness wint en toegevoegd wordt aan de parent populatie
    :param pop: De populatie die gebruikt wordt om parents van te selecteren
    :param size: De grootte van de parent populatie
    :param tournament_size: De grootte van een toernooi. Moet kleiner zijn dan population_size
    :return: de parent populatie
    """
    parent_pop = []
    for i in range(size):
        # selecteer tournament_size individuals (random sampling without replacement)
        tournament = rd.sample(pop, tournament_size)
        # selecteer de beste individual
        winner = np.argmin([i.fitness for i in tournament])
        # voeg deze toe aan parent_pop
        parent_pop.append(pop[winner])
    return(parent_pop)

Vervolgens is het tijd om nieuwe kinderen te creëren. Hieronder gebruiken we crossover: Een gedeelte van de ene ouder wordt gebruikt, en een ander gedeelte van de ander (zie hieronder).

![alt text](https://cdn-images-1.medium.com/max/1600/1*YhmzBBCyAG3rtEBbI0gz4w.jpeg)

Ook hier zijn er natuurlijk andere opties:

*   Single point crossover
*   k-point crossover
*   Uniform crossover
*   Edge recombination



In [0]:
def CrossOver(parent1, parent2, k, l):
    """
    crossover toegepast op twee parents
    :param parent1: parent 1
    :param parent2: parent 2
    :param k l: breakpoints van de genen. Voor k wordt gebruikt van parent1, daarna van parent2
    :return: een kind met gecombineerde genen
    """

    child = []
    # Selecteer de te-gebruiken genen van parent 1 & 2
    childcenter = [parent1.genes[j] for j in range(k,l+1)]
    childedges = [parent2.genes[j] for j in range(0,N) if parent2.genes[j] not in childcenter]
    # Voeg de genen samen
    child[:k] = childedges[:k]
    child[k:l+1] = childcenter
    child[l+1:] = childedges[k:]

    return(child)

Om nieuwe gebieden te ontdekken, kunnen we mutatie toevoegen op de kinderen. Hieronder passen we een swap operatie toe.

![alt text](http://www.wardsystems.com/manuals/genehunter/_bm53.png)

Twee random geselecteerde genen worden omgewisseld. Ook hier zijn er andere mogelijkheden, zoals:


*   Bit flip mutation
*   Uniform mutation
*   Gaussian mutation
*   Bit string mutation
*   Random Resetting
*   Insert
*   Scramble
*   Inversion
*   2-opt
*   k-opt

Hier is het ook afhankelijk van de representatie welke je kan gebruiken. Daarnaast ligt ook hier aan de aard van het probleem welke mutaties goed werken en welke slecht



In [0]:
def Swap(child, factor):                                   
    """
    Wissel twee genen om
    :param child: child to be mutated
    :param factor: in (1,2), worden er een of twee genen omgewisseld?
    :return: mutated child
    """

    for i in range(factor):
        (k, l) = rd.sample(range(N), 2)

        # swap two genes
        old = child[k]
        child[k] = child[l]
        child[l] = old

    return(child)

Zo kunnen er een heleboel kinderen gecreëerd worden. Uiteraard zullen sommige kinderen een lagere fitness hebben dan hun ouders, maar sommige zullen ook een hogere fitness hebben. Door het survival of the fittest principe toe te passen, maken betere kinderen een hogere kans om te herproduceren, waardoor op de lange termijn de hele populatie beter zal worden en sterkere genen zullen overleven.

Het is hierin belangrijk een goede afweging te maken tussen "exploration" en "optimization". Een hoge mutatiefactor en veel crossover zal zorgen dat er veel nieuwe gebieden ontdekt worden, maar als de selectiedruk te laag is (minder fitte individuen krijgen een te hoge kans om te herproduceren), zal de populatie niet beter worden. Daar tegenover staat als de selectiedruk te hoog is, het algoritme te snel in een lokaal optimum terecht kan komen, waardoor andere goede oplossingen gemist kunnen worden.

Hieronder staat de hele evolutionaire cyclus beschreven. Het algoritme stopt als de fitness van de beste individu 0 is, dus als er een oplossing is bereikt.

In [0]:
def GetBest(population):
    """
    :param: de huidige populatie
    :return: return de beste individual van de huidige populatie
    """
    best = np.argmin([i.fitness for i in population])
    return(population[best])

# Global parameters
tournament_size = 20 # De grootte van een toernooi
parent_size = 500 # De grootte van de parent set
mutation_factor = 0.8 # De kans op een enkele mutatie
mutation_factor_double = 0.5 # De kans op een dubbele mutatie, gegeven enkele mutatie

# Evolutionaire cyclus
i = 0
start_time = time.time()
while GetBest(population).fitness > 0:
    if i%10 == 0:
        print("iteratie", i)
    # Selecteer parents
    parents = TournamentSelection(population, parent_size, tournament_size)
    childs = []
    # Voeg het beste individu van de vorige populatie toe (Elitism)
    childs.append(GetBest(population))
    while len(childs) < population_size:
        # Selecteer parents uit de parent set
        parent1 = rd.sample(parents, 1)[0]
        parent2 = rd.sample(parents, 1)[0]
        # Maak twee kinderen (tegenovergestelde van elkaar) 
        (a,b) = rd.sample(range(N),2)
        k = min(a,b)
        l = max(a,b)
        child1 = CrossOver(parent1, parent2, k, l)
        child2 = CrossOver(parent2, parent1, k ,l)

        if bool(np.random.binomial(1, mutation_factor)):
            if bool(np.random.binomial(1, mutation_factor_double)):
                # Dubbele mutatie
                child1 = Swap(child1, 2)
            else:
                # Enkele mutatie
                child1 = Swap(child1, 1)
        if bool(np.random.binomial(1, mutation_factor)):
            if bool(np.random.binomial(1, mutation_factor_double)):
                # Dubbele mutatie
                child2 = Swap(child2, 2)
            else:
                # Enkele mutatie
                child2 = Swap(child2, 1)
        # Voeg kinderen toe
        childs.append(Individual(N, genes= child1))
        childs.append(Individual(N, genes= child2))

    # Update de populatie
    population = childs
    # Bereken de fitness voor de nieuwe populatie
    [i.GetFitness(N) for i in population]
    i = i + 1

end_time = time.time() - start_time
print("Oplossing gevonden")

iteratie 0
iteratie 10
iteratie 20
iteratie 30
iteratie 40
Oplossing gevonden


En tot slot: Print de oplossing

In [0]:
print("Oplossing gevonden: ", GetBest(population).genes)
print("After " + str(i) + " iterations and " + str(end_time) + " seconds.")

Oplossing gevonden:  [12, 3, 11, 8, 1, 5, 2, 10, 6, 0, 9, 4, 13, 7]
After 48 iterations and 5.596699237823486 seconds.


De code hierboven werkt dus redelijk snel als je bedenkt dat er N! configuraties zijn. Door te spelen met de parameter settings zou het wellicht nog sneller kunnen.

Hierboven is de basis van een evolutionair algoritme beschreven. Uiteraard zijn er veel extensies en aanpassingen mogelijk. Te denken valt aan

*   Parameters kunnen veranderen over tijd
*   Parameters kunnen ook mee evalueren
*   Waarom zou je jezelf limiteren aan de wetten van de natuur? 3 of meer ouders kan op een computer ook prima
*   Verschillende crossover en mutatie methodes in hetzelfde algoritme
*   enz.


